In [ ]:
import pandas as pd
import requests
import json
from datetime import datetime
import time

def obtener_datos_nasa(latitud, longitud, fecha):
    """
    Obtiene datos climáticos de la NASA POWER API para una ubicación y fecha específicas
    """
    try:
        # Convertir fecha a formato YYYYMMDD
        if isinstance(fecha, str):
            fecha_obj = datetime.strptime(fecha, "%Y-%m-%d")
        else:
            fecha_obj = fecha
        fecha_formateada = fecha_obj.strftime("%Y%m%d")
        
        # Parámetros de la API
        base_url = "https://power.larc.nasa.gov/api/temporal/daily/point"
        parametros = "T2M,PRECTOTCORR,ALLSKY_SFC_SW_DWN,CLOUD_AMT"
        
        # Construir URL
        url = f"{base_url}?parameters={parametros}&community=AG&longitude={longitud}&latitude={latitud}&start={fecha_formateada}&end={fecha_formateada}&format=JSON"
        
        # Hacer solicitud a la API
        response = requests.get(url, timeout=30)
        response.raise_for_status()
        
        datos = response.json()
        
        # Extraer los datos específicos
        propiedades = datos['properties']['parameter']
        geometria = datos['geometry']
        
        # Formatear los datos según especificaciones
        temperatura = round(propiedades['T2M'][fecha_formateada], 1)
        nubosidad = round(propiedades['CLOUD_AMT'][fecha_formateada], 1)
        radiacion_solar = round(propiedades['ALLSKY_SFC_SW_DWN'][fecha_formateada], 2)
        precipitacion = round(propiedades['PRECTOTCORR'][fecha_formateada], 1)
        
        return {
            'temperatura_promedio': temperatura,
            'nubosidad': nubosidad,
            'radiacion_solar': f"{radiacion_solar} MJ/m²",
            'precipitacion': f"{precipitacion} mm",
            'geometria': json.dumps(geometria)
        }
        
    except Exception as e:
        print(f"Error obteniendo datos para lat {latitud}, lon {longitud}, fecha {fecha}: {e}")
        return None

def procesar_ubicaciones():
    """
    Procesa el CSV de ubicaciones y agrega los datos de NASA POWER
    """
    try:
        # Leer el CSV
        df = pd.read_csv('ubicaciones.csv')
        
        # Verificar y agregar columnas si no existen
        columnas_necesarias = ['temperatura_promedio', 'nubosidad', 'radiacion_solar', 'precipitacion', 'geometria']
        for columna in columnas_necesarias:
            if columna not in df.columns:
                df[columna] = None
        
        # Procesar cada fila
        for index, fila in df.iterrows():
            # Saltar si ya tiene datos
            if pd.notna(fila['temperatura_promedio']) and pd.notna(fila['nubosidad']):
                continue
            
            print(f"Procesando fila {index + 1}/{len(df)}...")
            
            # Obtener datos de la fila actual
            latitud = fila['latitud']
            longitud = fila['longitud']
            fecha = fila['fecha']
            
            # Obtener datos de NASA
            datos_nasa = obtener_datos_nasa(latitud, longitud, fecha)
            
            if datos_nasa:
                # Actualizar los datos en el DataFrame
                for clave, valor in datos_nasa.items():
                    df.at[index, clave] = valor
                
                # Guardar inmediatamente después de cada actualización
                df.to_csv('ubicaciones.csv', index=False)
                print(f"✓ Datos actualizados para fila {index + 1}")
            else:
                print(f"✗ No se pudieron obtener datos para fila {index + 1}")
            
            # Pequeña pausa para no saturar la API
            time.sleep(1)
            
        print("Procesamiento completado!")
        
    except Exception as e:
        print(f"Error en el procesamiento: {e}")

# Ejecutar el procesamiento
if __name__ == "__main__":
    procesar_ubicaciones()

Procesando fila 1/9999...
✓ Datos actualizados para fila 1
Procesando fila 2/9999...
✓ Datos actualizados para fila 2
Procesando fila 3/9999...
✓ Datos actualizados para fila 3
Procesando fila 4/9999...
✓ Datos actualizados para fila 4
Procesando fila 5/9999...
✓ Datos actualizados para fila 5
Procesando fila 6/9999...
✓ Datos actualizados para fila 6
Procesando fila 7/9999...
✓ Datos actualizados para fila 7
Procesando fila 8/9999...
✓ Datos actualizados para fila 8
Procesando fila 9/9999...
✓ Datos actualizados para fila 9
Procesando fila 10/9999...
✓ Datos actualizados para fila 10
Procesando fila 11/9999...
✓ Datos actualizados para fila 11
Procesando fila 12/9999...
✓ Datos actualizados para fila 12
Procesando fila 13/9999...
✓ Datos actualizados para fila 13
Procesando fila 14/9999...
✓ Datos actualizados para fila 14
Procesando fila 15/9999...
✓ Datos actualizados para fila 15
Procesando fila 16/9999...
✓ Datos actualizados para fila 16
Procesando fila 17/9999...
✓ Datos actuali